## Using Komo for IK

In [1]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry

In [2]:
# Here we do not need a simulation world
# adding a configuration world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
V = ry.ConfigurationViewer()
V.setConfiguration(C)

In [3]:
# Add a ssbox
obj = C.addFrame("object")
obj.setPosition([0,0.2,1.5])
obj.setQuaternion([1,0.3,.4,0.1])
obj.setShape(ry.ST.ssBox, size=[.2, .2, .2, .02])
obj.setColor([1,1,0])
V.setConfiguration(C)

In [4]:
# Set contacts for coliision handling

#Contact for table
Table = C.frame("table")
Table.setContact(1)

# Contact for Right gripper and fingers
R_gripper = C.frame("R_gripper")
R_gripper.setContact(1)
R_finger1 = C.frame("R_finger1")
R_finger1.setContact(1)
R_finger2 = C.frame("R_finger2")
R_finger2.setContact(1)


# Contact for Left gripper
L_gripper = C.frame("L_gripper")
L_gripper.setContact(1)

# Contact for object
obj.setContact(1)

In [5]:
# Get initial position
[yli,Jli] = C.evalFeature(ry.FS.position, ["L_gripperCenter"])
[yri,Jri] = C.evalFeature(ry.FS.position, ["R_gripperCenter"])

**1a**

In [5]:
#1a
IK = C.komo_IK(False)

# addObjective(times, featureSymbol, frameNames, objectiveType, scale, target, order)
# To prevent collisions
IK.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])

# Position difference between gripper centre is 0
IK.addObjective([1], ry.FS.positionDiff, ['R_gripperCenter', 'L_gripperCenter'], ry.OT.sos, [1e2])

# Both X axis orthogonal
IK.addObjective([1], ry.FS.scalarProductXX, ['R_gripperCenter', 'L_gripperCenter'], ry.OT.sos, [1e2])
# IK.addObjective([1.], ry.FS.scalarProductXX, ['R_gripperCenter', 'L_gripperCenter'], ry.OT.eq, [1e2])

# Both Z axis are anti parallel, for opposite facing
IK.addObjective([1.], ry.FS.scalarProductZZ, ['R_gripperCenter', 'L_gripperCenter'], ry.OT.sos, [1e2], target=[-1.])
# IK.addObjective([1.], ry.FS.scalarProductZZ, ['R_gripperCenter', 'L_gripperCenter'], ry.OT.eq, [1e2], target=[-1.])


**1b**

In [44]:
#1b
# move object
obj.setPosition([0,.2,1.5])

IK = C.komo_IK(False)
# addObjective(times, featureSymbol, frameNames, objectiveType, scale, target, order)
# To prevent collisions
IK.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])

# Position difference between gripper centre is 0
IK.addObjective([0], ry.FS.positionDiff, ['R_gripperCenter', 'object'], ry.OT.eq, [1e0])

# Align with respect to any of the axis (Nearest axis)
IK.addObjective([1], ry.FS.scalarProductXX, ['R_gripperCenter', 'object'], ry.OT.eq, [1e2])
# IK.addObjective([1.], ry.FS.scalarProductXX, ['R_gripperCenter', 'L_gripperCenter'], ry.OT.eq, [1e2])

# Add distance specifically to prevent collisions
#IK.addObjective([1], ry.FS.distance, ['R_gripper', 'object'], ry.OT.eq)
#IK.addObjective([], ry.FS.distance, ['R_finger1', 'object'], ry.OT.ineq)
#IK.addObjective([], ry.FS.distance, ['R_finger2', 'object'], ry.OT.ineq)


In [7]:
# Calling the optimizer (True means random initialization/restart)
IK.optimize(True)
IK.getReport()

[{'x_dim': 16, 'T': 1, 'k_order': 1, 'tau': 1.0, 'useSwift': False},
 {'order': 1.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [-1, 0],
  'sos_sumOfSqr': 0.057025174823319036},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [0],
  'sos_sumOfSqr': 0.0005702517482331904},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'QuaternionNorms',
  'vars': [0],
  'eq_sumOfAbs': 0.0},
 {'order': 0.0,
  'type': 'ineq',
  'feature': 'ProxyCost',
  'vars': [0],
  'inEq_sumOfPos': 0.0},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'Default-0-posDiff-R_gripperCenter-L_gripperCenter',
  'vars': [0],
  'sos_sumOfSqr': 6.631589885000414e-06},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'Default-0-vecAlign-R_gripperCenter-L_gripperCenter',
  'vars': [0],
  'sos_sumOfSqr': 3.2860004599875536e-06},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'Default-0-vecAlign-R_gripperCenter-L_gripperCenter',
  'vars': [0],
  'sos_sumOfSqr': 0.00024229362014940452}]

In [46]:
C.setFrameState( IK.getConfiguration(0) )
V.setConfiguration(C) #to update your model display

In [43]:
# Set the arms to random positions
IK = C.komo_IK(False)
IK.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])
IK.addObjective([1.], ry.FS.position, ["L_gripperCenter"], ry.OT.eq, [1e2], target=yli)
IK.addObjective([1.], ry.FS.position, ["R_gripperCenter"], ry.OT.eq, [1e2], target=yri)

# Calling the optimizer (True means random initialization/restart)
IK.optimize(True)
IK.getReport()

C.setFrameState( IK.getConfiguration(0) )
V.setConfiguration(C) #to update your model display

## Path Optimization

**1a**

In [16]:
# 1a
# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
komo = C.komo_path(1.,20, 5., True) 

# addObjective(times, featureSymbol, frameNames, objectiveType, scale, target, order)
# To prevent collisions
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])

# Position difference between gripper centre is 0
komo.addObjective([1], ry.FS.positionDiff, ['R_gripperCenter', 'L_gripperCenter'], ry.OT.sos, [1e2])

# Both X axis orthogonal
komo.addObjective([1], ry.FS.scalarProductXX, ['R_gripperCenter', 'L_gripperCenter'], ry.OT.sos, [1e2])
# IK.addObjective([1.], ry.FS.scalarProductXX, ['R_gripperCenter', 'L_gripperCenter'], ry.OT.eq, [1e2])

# Both Z axis are anti parallel, for opposite facing
komo.addObjective([1.], ry.FS.scalarProductZZ, ['R_gripperCenter', 'L_gripperCenter'], ry.OT.sos, [1e2], target=[-1.])

komo.optimize(True)
komo.getReport()

[{'x_dim': 320, 'T': 20, 'k_order': 2, 'tau': 0.25, 'useSwift': True},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'QuaternionNorms',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'eq_sumOfAbs': 0.0},
 {'order': 2.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 0.2002868135156707},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 0.0007528270391629072},
 {'order': 0.0,
  'type': 'ineq',
  'feature': 'ProxyCost',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'inEq_sumOfPos': 0.024322078898837002},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'Default-0-posDiff-R_gripperCenter-L_gripperCenter',
  'vars': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  'sos_sumOfSqr': 9.346400875162338e-05},
 {'order': 0.0,
  'type': 'sos',
  'fe

**1b**

In [6]:
#1b
# move object
obj.setPosition([0,.2,1.5])

# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
komo = C.komo_path(1.,20, 5., True) 

# addObjective(times, featureSymbol, frameNames, objectiveType, scale, target, order)
# To prevent collisions
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])

# Position difference between gripper centre is 0
komo.addObjective([], ry.FS.positionDiff, ['R_gripperCenter', 'object'], ry.OT.sos, [1e2])

# Align with respect to any of the axis (Nearest axis)
komo.addObjective([1], ry.FS.scalarProductXX, ['R_gripperCenter', 'object'], ry.OT.eq, [1e2])
# komo.addObjective([1.], ry.FS.scalarProductXX, ['R_gripperCenter', 'L_gripperCenter'], ry.OT.eq, [1e2])

komo.optimize(True)
komo.getReport()

[{'x_dim': 320, 'T': 20, 'k_order': 2, 'tau': 0.25, 'useSwift': True},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'QuaternionNorms',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'eq_sumOfAbs': 0.0},
 {'order': 2.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 148.95370067066978},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 0.006271462709905664},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'Default-0-posDiff-R_gripperCenter-object',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 46.99744296830443},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'Default-0-vecAlign-R_gripperCenter-object',
  'vars': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  'eq_sumOfAbs': 6.95584998822163e-05}]

In [7]:
V = komo.view()

In [8]:
V.playVideo()

In [9]:
C.setFrameState(komo.getConfiguration(19))
C.getJointState()
V.setConfiguration(C)
coll = C.feature(ry.FS.accumulatedCollisions, [])
C.computeCollisions() 
coll.eval(C)

RuntimeError: /home/muddasser/git/dev/robotics-course/rai/rai/Geo/pairCollision.cpp:libccd:259(-2) simplex types 4 4 not handled